In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Output')
cur_date = "041822"

library(readxl)
library(tidyverse)


#reading in file
proteomics_df = data.frame(read_excel("041822_Imputed_Proteomics_Data.xlsx"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
head(proteomics_df)

,ID,Set,Treatment,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,CoCulture_Exposed1,Set1,CoCulture_Exposed,P19012,476617.6
2,CoCulture_Exposed1,Set1,CoCulture_Exposed,P19013,1077007.5
3,CoCulture_Exposed1,Set1,CoCulture_Exposed,O43897,2245997.6
4,CoCulture_Exposed1,Set1,CoCulture_Exposed,P20700,2388716.6
5,CoCulture_Exposed1,Set1,CoCulture_Exposed,P22102,1790626.6
6,CoCulture_Exposed1,Set1,CoCulture_Exposed,O75367,874143.2


In [3]:
proteomics_df = proteomics_df %>%
    #for some reason protein "Q9UDY2" is only in the vehicle group, so removing it
    #filter(Protein != "Q14117") %>%
    #just want a column that specifies the treatment and exposure separately
    separate(Treatment, c("Treatment", "Exposure"), sep = "_") 

head(proteomics_df)

,ID,Set,Treatment,Exposure,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,CoCulture_Exposed1,Set1,CoCulture,Exposed,P19012,476617.6
2,CoCulture_Exposed1,Set1,CoCulture,Exposed,P19013,1077007.5
3,CoCulture_Exposed1,Set1,CoCulture,Exposed,O43897,2245997.6
4,CoCulture_Exposed1,Set1,CoCulture,Exposed,P20700,2388716.6
5,CoCulture_Exposed1,Set1,CoCulture,Exposed,P22102,1790626.6
6,CoCulture_Exposed1,Set1,CoCulture,Exposed,O75367,874143.2


In [9]:
#explain why I split this
proteomics_df_split = proteomics_df %>%
    group_by(Set) %>%
    group_split()

set1_proteomics_df = proteomics_df_split[[1]]
set2_proteomics_df = proteomics_df_split[[2]]

head(set1_proteomics_df)

ID,Set,Treatment,Exposure,Protein,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
CoCulture_Exposed1,Set1,CoCulture,Exposed,P19012,476617.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P19013,1077007.5
CoCulture_Exposed1,Set1,CoCulture,Exposed,O43897,2245997.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P20700,2388716.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P22102,1790626.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,O75367,874143.2


Running t tests to determine if there are statistically significant distribution differences in co-culture unexposed vs. exposed and tri-culture unexposed vs. exposed. 

In [13]:
t_test_values = function(df, set_id){
    #"""
    #Running t tests after filtering for set, treatment, exposure, and protein using a loop. 
    #Ultimately using this test to compare proteins (unexposed vs. exposed).

    #:param: subsetted dataframe, empty dataframe
    #:output: a dataframe containing the set, treatment, exposure, protein, u stat, p value, p adj

    #"""
    treatments = unique(df$Treatment)
    proteins = unique(df$Protein)
    
    values_df = data.frame()
    #iterating through each tx, protein

    for(i in 1:length(treatments)){
        for(j in 1:length(proteins)){
            #unexposed df
            unexposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Vehicle", Protein == proteins[j])
            #exposed df
            exposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Exposed", Protein == proteins[j])

            #t test
            t_test = t.test(unexposed_df$Value, exposed_df$Value)

            #contains smoking status compared, compartment, cytokine, u stat, and p value
            values_vector = cbind(set_id, treatments[i], proteins[j], t_test$statistic, t_test$p.value)
            values_df = rbind(values_df, values_vector)
            }
        }

    
    #adding col names
    colnames(values_df) = c("Set", "Treatment", "Protein", "Statistic", "P Value")
    
    
    #calculating padj values
    
    padj_vector = c()
    for (i in 1:length(treatments)){
        `P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")
        padj_vector = c(padj_vector, `P Adj`)
    }
    
    values_df = cbind(values_df, padj_vector) %>%
        rename(`P Adj` = padj_vector)
    
    return(values_df)
}

In [15]:
#calling fn
set1_t_test = t_test_values(set1_proteomics_df, "Set1")
set2_t_test = t_test_values(set2_proteomics_df, "Set2")

,Set,Treatment,Protein,Statistic,P Value,padj_vector
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
t,Set1,CoCulture,P19012,0.60331489149206,0.567224863429138,0.8207093
t1,Set1,CoCulture,P19013,-0.139436949314188,0.893842444121871,0.9672176
t2,Set1,CoCulture,O43897,-1.28817058222099,0.241597197697623,0.7149305
t3,Set1,CoCulture,P20700,-0.401101776101729,0.698456072213535,0.8649296
t4,Set1,CoCulture,P22102,1.09076873867442,0.311878044109208,0.7329387
t5,Set1,CoCulture,O75367,-0.603646331227886,0.563303637544858,0.8186874


,Set,Treatment,Protein,Statistic,P Value,padj_vector
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
t,Set2,TriCulture,Q7Z3Y8,1.4572273303619,0.177587222183449,0.8730653
t1,Set2,TriCulture,Q6KB66.1,1.12131924845407,0.312792449034433,0.9992881
t2,Set2,TriCulture,Q14117,0.27614424008717,0.788439221107136,0.9992881
t3,Set2,TriCulture,P22223,-0.148301469930139,0.885064163315534,0.9992881
t4,Set2,TriCulture,P05976,1.04028515484624,0.33787297883861,0.9992881
t5,Set2,TriCulture,Q9NZT1,1.03714180050857,0.34708499993666,0.9992881


In [23]:
# creating 1 df
t_test_df = rbind(set1_t_test, set2_t_test)
head(t_test_df)

ID,Set,Treatment,Exposure,Protein,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>


ID,Set,Treatment,Exposure,Protein,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>


ERROR: Error in t.test.default(a$Value, b$Value): not enough 'x' observations


In [ ]:
a$